In [2]:
import sys
sys.path.insert(1, '../../libs')
from utils import get_data, temporal_train_test_split, evaluate_keras

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate, train_test_split
from tensorflow import keras
from nltk.tokenize import TweetTokenizer

In [3]:
data = get_data("../../data/authors.csv")

In [4]:
tt = TweetTokenizer()

vectorizers = [CountVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,5), analyzer="char"),
                CountVectorizer(ngram_range=(4,5), analyzer="char"),
                CountVectorizer(ngram_range=(3,8), analyzer="char"),
                TfidfVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(4,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(3,8), analyzer="char")]

In [5]:
def build_model(input_shape, output_shape):  
    model = keras.models.Sequential(name="NeuralNetwork")
    model.add(keras.layers.Input(shape=input_shape, name="Input"))
    model.add(keras.layers.Dense(300, activation="relu", name="Dense1"))
    model.add(keras.layers.Dense(300, activation="relu", name="Dense2"))
    model.add(keras.layers.Dense(300, activation="relu", name="Dense3"))
    model.add(keras.layers.Dense(output_shape, activation="softmax", name="Output"))
    return model

In [7]:
evaluation = list()
usernames = list(np.unique(data["username"]))
results = list()

for i, vectorizer in enumerate(vectorizers):
    vectorizer_str = vectorizer.__str__()
    print(f"Running => {vectorizer_str}")

    for i in range(len(usernames)):
        author1 = usernames.pop()

        for author2 in usernames:
            X_train, X_test, y_train, y_test = temporal_train_test_split(
                data, author1, author2)

            scaler = MinMaxScaler()
            X_train = vectorizer.fit_transform(X_train["comment"]).toarray()
            X_test = vectorizer.transform(X_test["comment"]).toarray()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            y_classes = pd.get_dummies(y_train).columns
            y_train = pd.get_dummies(y_train).values
            y_test = pd.get_dummies(y_test).values
            
            input_shape_text = X_train.shape[1]
            output_shape = y_train.shape[1]

            callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

            model = build_model(input_shape_text, output_shape)
            model.compile(loss = "categorical_crossentropy",
                        optimizer = keras.optimizers.SGD(learning_rate=0.01),
                        metrics = ["accuracy"])
            history = model.fit(X_train, y_train, epochs=1000, callbacks=[callback], validation_split=0.1, shuffle=True, verbose=True)


            y_pred_proba = model.predict(X_test)
            evaluation.append(evaluate_keras(y_test.argmax(1), y_pred_proba, *y_classes))
            metrics = pd.DataFrame(evaluation)[["f1_macro", "recall_macro", "precision_macro", "accuracy", "auc_score"]].mean()
            metrics["vectorizer"] = [vectorizer_str for i in range(len(metrics))]
            print("==================")
            print("F1:", metrics["f1_macro"])
            print("Acc:", metrics["accuracy"])
            print("AUC:", metrics["auc_score"])
            print("==================")
        break
    break
    results.append(metrics)

Running => CountVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f187507c0d0>>)
Epoch 1/1000
42/42 [==============================] - 1s 14ms/step - loss: 0.6933 - accuracy: 0.4732 - val_loss: 0.6923 - val_accuracy: 0.4830
Epoch 2/1000
42/42 [==============================] - 0s 12ms/step - loss: 0.6917 - accuracy: 0.5117 - val_loss: 0.6912 - val_accuracy: 0.5578
Epoch 3/1000
42/42 [==============================] - 0s 11ms/step - loss: 0.6902 - accuracy: 0.5268 - val_loss: 0.6904 - val_accuracy: 0.5918
Epoch 4/1000
42/42 [==============================] - 0s 10ms/step - loss: 0.6887 - accuracy: 0.5979 - val_loss: 0.6894 - val_accuracy: 0.5850
Epoch 5/1000
42/42 [==============================] - 0s 11ms/step - loss: 0.6870 - accuracy: 0.6372 - val_loss: 0.6883 - val_accuracy: 0.6122
Epoch 6/1000
42/42 [==============================] - 1s 12ms/step - loss: 0.6853 - accuracy: 0.6538 - val_loss: 0.6872 - val_accuracy: 0.6190

In [ ]:
results

In [ ]:
metrics_df = pd.DataFrame([results[i][1] for i in range(len(results))])
metrics_df["vectorizer"] = [results[i][0] for i in range(len(results))]
metrics_df.to_csv("../../results/neural_network.csv")
metrics_df